In [1]:
import pandas as pd
from prepare import Prepare
from split_get_scale import SplitGetScale
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso, Ridge, TweedieRegressor
from sklearn.cluster import KMeans

pd.options.display.max_columns = 30

zillow = Prepare().wrangle_zillow()
zillow.head()

/Users/dragonzord/Documents/SchoolDocs/codeup-data-science/exercises/methodologies1/clustering/zillow_clustering/prepare.py:11: DtypeWarning: Columns (63) have mixed types.Specify dtype option on import or set low_memory=False.
  df = Acquire().get_zillow_data()


,parcel_id,bathroom_count,bedroom_count,home_size_square_feet,fips,latitude,longitude,lot_size_square_feet,raw_census_tract_and_block,region_id_city,region_id_county,region_id_zip,year_built,structure_tax_value_price,tax_value_dollar_count,assessment_year,land_tax_value_price,tax_amount,census_tract_and_block,log_error,transaction_date,county
0,14297519,3.5,4,3100.0,6059,33634931.0,-117869207.0,4506.0,6.059063e+07,53571,1286,96978,1998,485713.0,1023282.0,2016,537569.0,11013.72,6.059063e+13,0.025595,2017-01-01,Orange
1,17052889,1.0,2,1465.0,6111,34449266.0,-119281531.0,12647.0,6.111001e+07,13091,2061,97099,1967,88000.0,464000.0,2016,376000.0,5672.48,6.111001e+13,0.055619,2017-01-01,Ventura
2,14186244,2.0,3,1243.0,6059,33886168.0,-117823170.0,8432.0,6.059022e+07,21412,1286,97078,1962,85289.0,564778.0,2016,479489.0,6488.30,6.059022e+13,0.005383,2017-01-01,Orange
3,12177905,3.0,4,2376.0,6037,34245180.0,-118240722.0,13038.0,6.037300e+07,396551,3101,96330,1970,108918.0,145143.0,2016,36225.0,1777.51,6.037300e+13,-0.103410,2017-01-01,Los Angeles
4,10887214,3.0,3,1312.0,6037,34185120.0,-118414640.0,278581.0,6.037124e+07,12447,3101,96451,1964,73681.0,119407.0,2016,45726.0,1533.89,6.037124e+13,0.006940,2017-01-01,Los Angeles


In [2]:
sgs = SplitGetScale()
train, validate, test = sgs.split(zillow)
train.head()

,parcel_id,bathroom_count,bedroom_count,home_size_square_feet,fips,latitude,longitude,lot_size_square_feet,raw_census_tract_and_block,region_id_city,region_id_county,region_id_zip,year_built,structure_tax_value_price,tax_value_dollar_count,assessment_year,land_tax_value_price,tax_amount,census_tract_and_block,log_error,transaction_date,county
13527,11770180,1.0,2,941.0,6037,34012117.0,-118247150.0,6338.0,6.037228e+07,12447,3101,95992,1909,32430.0,72079.0,2016,39649.0,1025.09,6.037228e+13,-0.091213,2017-03-01,Los Angeles
52447,10884378,3.0,3,1560.0,6037,34203300.0,-118430000.0,7313.0,6.037127e+07,12447,3101,96414,2007,166541.0,363890.0,2016,197349.0,4473.98,6.037127e+13,-0.031320,2017-06-30,Los Angeles
2559,11749839,1.0,5,2986.0,6037,34048146.0,-118311705.0,12490.0,6.037213e+07,12447,3101,96000,1921,43056.0,78654.0,2016,35598.0,1193.71,6.037213e+13,0.059301,2017-01-12,Los Angeles
33056,10906993,2.0,1,1271.0,6037,34151000.0,-118406000.0,19158.0,6.037143e+07,12447,3101,96449,1978,151657.0,469812.0,2016,318155.0,5630.53,6.037143e+13,-0.021502,2017-05-04,Los Angeles
63509,10903027,3.0,3,2043.0,6037,34160988.0,-118412064.0,5001.0,6.037125e+07,12447,3101,96452,1962,191127.0,312927.0,2016,121800.0,3887.88,6.037125e+13,0.048015,2017-08-03,Los Angeles


In [3]:
train.columns

Index(['parcel_id', 'bathroom_count', 'bedroom_count', 'home_size_square_feet',
       'fips', 'latitude', 'longitude', 'lot_size_square_feet',
       'raw_census_tract_and_block', 'region_id_city', 'region_id_county',
       'region_id_zip', 'year_built', 'structure_tax_value_price',
       'tax_value_dollar_count', 'assessment_year', 'land_tax_value_price',
       'tax_amount', 'census_tract_and_block', 'log_error', 'transaction_date',
       'county'],
      dtype='object')

In [4]:
cols_keep = ['parcel_id', 'bathroom_count', 'bedroom_count', 'home_size_square_feet',
            'fips', 'latitude', 'longitude', 'lot_size_square_feet',
            'raw_census_tract_and_block', 'region_id_city', 'region_id_county',
            'region_id_zip', 'year_built', 'structure_tax_value_price',
            'tax_value_dollar_count', 'assessment_year', 'land_tax_value_price',
            'tax_amount', 'census_tract_and_block']

In [5]:
train_scaled, val_scaled, test_scaled, _ = sgs.scale(train[cols_keep], validate[cols_keep], test[cols_keep])

In [6]:
#  features that will be used are 'bathroom_count', 'bedroom_count','home_size_square_feet', 'lot_size_square_feet'
home_features = ['bathroom_count', 'bedroom_count', 'home_size_square_feet', 'lot_size_square_feet']
clusters_train_scaled = train_scaled[home_features]
clusters_validate_scaled = val_scaled[home_features]
clusters_test_scaled = test_scaled[home_features]

In [7]:
# creating the object
kmeans = KMeans(n_clusters=5, max_iter=500)

# fitting the object
kmeans.fit(clusters_train_scaled)

#predicting using the kmeans object
y_kmeans_train = kmeans.predict(clusters_train_scaled)

In [8]:
# Appending cluster into the DataFrame
train['home_features_cluster'] = y_kmeans_train

In [9]:
#predicting using the kmeans object
y_kmeans_validate = kmeans.predict(clusters_validate_scaled)
validate['home_features_cluster'] = y_kmeans_validate

In [10]:
#predicting using the kmeans object
y_kmeans_test = kmeans.predict(clusters_test_scaled)
test['home_features_cluster'] = y_kmeans_test

In [11]:
(X_train, y_train), (X_val, y_val), (X_test, y_test) = sgs.get_Xy(train, validate, test, cols_dummy=["county", "home_features_cluster"])

### Baseline

In [12]:
act_pred_error = pd.DataFrame({"actual": y_train})
act_pred_error

,actual
13527,-0.091213
52447,-0.031320
2559,0.059301
33056,-0.021502
63509,0.048015
...,...
65697,-0.023112
9026,0.048371
37603,0.005313
47311,0.029311


In [13]:
act_pred_error["baseline_prediction"] = y_train.mean()
act_pred_error

,actual,baseline_prediction
13527,-0.091213,0.016579
52447,-0.031320,0.016579
2559,0.059301,0.016579
33056,-0.021502,0.016579
63509,0.048015,0.016579
...,...,...
65697,-0.023112,0.016579
9026,0.048371,0.016579
37603,0.005313,0.016579
47311,0.029311,0.016579


In [14]:
baseline_rmse = mean_squared_error(act_pred_error["actual"], act_pred_error["baseline_prediction"], squared=False)
baseline_rmse

0.16559772875837225

### Baseline RMSE is 0.16559772875837225

In [16]:
def grid_search(X, y, model, params_dic):
    grid = GridSearchCV(model, params_dic, n_jobs=-1)
    return grid.fit(X, y)

In [17]:
lasso_grid = {"alpha": [0.25, 0.5, 0.75, 1.0], "warm_start": [True, False]}
ridge_grid = {"alpha": [0.25, 0.5, 0.75, 1.0], }
tweedie_grid = {"power": [0], "alpha": [0.25, 0.5, 0.75, 1.0], "warm_start": [True, False]}

lasso = grid_search(X_train, y_train, Lasso(random_state=123), lasso_grid)
ridge = grid_search(X_train, y_train, Ridge(random_state=123), ridge_grid)
tweddie = grid_search(X_train, y_train, TweedieRegressor(), tweedie_grid)

In [19]:
print(lasso.best_estimator_)
print(lasso.best_params_)

print(ridge.best_estimator_)
print(ridge.best_params_)

print(tweddie.best_estimator_)
print(tweddie.best_params_)

Lasso(alpha=0.25, random_state=123, warm_start=True)
{'alpha': 0.25, 'warm_start': True}
Ridge(random_state=123)
{'alpha': 1.0}
TweedieRegressor(power=0, warm_start=True)
{'alpha': 1.0, 'power': 0, 'warm_start': True}


In [20]:
lasso = Lasso(alpha=0.25, random_state=123, warm_start=True).fit(X_train, y_train)
ridge = Ridge(alpha=1.0, random_state=123).fit(X_train, y_train)
tweedie = TweedieRegressor(alpha=1.0, power=0, warm_start=True).fit(X_train, y_train)
lr = LinearRegression().fit(X_train, y_train)

In [21]:
lass_pred_train = lasso.predict(X_train)
ridge_pred_train = ridge.predict(X_train)
tweed_pred_train = tweedie.predict(X_train)
lr_pred_train = lr.predict(X_train)

In [24]:
lass_pred_val = lasso.predict(X_val)
ridge_pred_val = ridge.predict(X_val)
tweed_pred_val = tweedie.predict(X_val)
lr_pred_val = lr.predict(X_val)

### Baseline RMSE is 0.16559772875837225

In [27]:
print(f"TRAIN\nlasso train rmse: {mean_squared_error(y_train, lass_pred_train, squared=False)}\nridge train rmse: {mean_squared_error(y_train, ridge_pred_train, squared=False)}\ntweedie train rmse: {mean_squared_error(y_train, tweed_pred_train, squared=False)}\nlr train rmse: { mean_squared_error(y_train, lr_pred_train, squared=False)}")

print(f"VALIDATE\nlasso validation rmse: {mean_squared_error(y_val, lass_pred_val, squared=False)}\nridge validation rmse: {mean_squared_error(y_val, ridge_pred_val, squared=False)}\ntweedie validation rmse: {mean_squared_error(y_val, tweed_pred_val, squared=False)}\nlr validation rmse: { mean_squared_error(y_val, lr_pred_val, squared=False)}")

TRAIN
lasso train rmse: 0.16535618032732577
ridge train rmse: 0.1653104033608303
tweedie train rmse: 0.1653544702677273
lr train rmse: 0.16531040332584376
VALIDATE
lasso validation rmse: 0.1677281814345548
ridge validation rmse: 0.1677076727075786
tweedie validation rmse: 0.16773395814835557
lr validation rmse: 0.16770767342966553
